In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv("D:/Code_ML/dynamic_pricing_engine/data/processed/simulated_pricing_data.csv", parse_dates=['Date'])
# parse dates converts the 'Date' column to datetime64 object (datatype)

In [38]:
df.head()

,Product_ID,Date,Base_Cost,Price,Competitor_Price,Inventory,Units_Demanded
0,P1,2024-01-01,15.48,28.13,29.41,46,106
1,P1,2024-01-02,15.48,28.94,28.40,156,78
2,P1,2024-01-03,15.48,27.54,27.36,178,112
3,P1,2024-01-04,15.48,28.00,30.52,143,114
4,P1,2024-01-05,15.48,27.57,28.48,98,122


In [39]:
# Competitor price delta
df['Competitor_Delta'] = df['Competitor_Price'] - df['Price']

In [40]:
# Margin
df['Margin'] = df['Price'] - df['Base_Cost']

In [41]:
# Price to cost ratio
df['Price_Cost_Ratio'] = df['Price'] / df['Base_Cost']

In [42]:
# Inventory pressure flag (stock out risk)
df["Inventory_Pressure"] = df["Inventory"].apply(lambda x: 1 if x < 30 else 0)

In [43]:
# Day Month Week Features
df["Day_Week"] = df["Date"].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Week_Year'] = df['Date'].dt.isocalendar().week.astype(int)  # Convert to int type

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18250 entries, 0 to 18249
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product_ID          18250 non-null  object        
 1   Date                18250 non-null  datetime64[ns]
 2   Base_Cost           18250 non-null  float64       
 3   Price               18250 non-null  float64       
 4   Competitor_Price    18250 non-null  float64       
 5   Inventory           18250 non-null  int64         
 6   Units_Demanded      18250 non-null  int64         
 7   Competitor_Delta    18250 non-null  float64       
 8   Margin              18250 non-null  float64       
 9   Price_Cost_Ratio    18250 non-null  float64       
 10  Inventory_Pressure  18250 non-null  int64         
 11  Day_Week            18250 non-null  int32         
 12  Month               18250 non-null  int32         
 13  Week_Year           18250 non-null  int64     

In [45]:
# Rolling Demand Features

df['rolling_demand_7'] = df.groupby('Product_ID')['Units_Demanded'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())


In [46]:
df['rolling_demand_30'] = df.groupby('Product_ID')['Units_Demanded'].transform(lambda x: x.rolling(window=30, min_periods=1).mean())

In [47]:
# Lag Demand Features
df['lag_1'] = df.groupby('Product_ID')['Units_Demanded'].shift(1)
df['lag_7'] = df.groupby('Product_ID')['Units_Demanded'].shift(7)
df['lag_30'] = df.groupby('Product_ID')['Units_Demanded'].shift(30)

In [48]:
# Fill NA Values
df.fillna(0, inplace=True)

In [49]:
df.describe()

,Date,Base_Cost,Price,Competitor_Price,Inventory,Units_Demanded,Competitor_Delta,Margin,Price_Cost_Ratio,Inventory_Pressure,Day_Week,Month,Week_Year,rolling_demand_7,rolling_demand_30,lag_1,lag_7,lag_30
count,18250,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000,18250.000000
mean,2024-07-01 00:00:00.000000256,26.217600,42.657650,42.676384,109.663781,99.507781,0.018733,16.440050,1.626430,0.056932,2.991781,6.498630,26.430137,99.566491,99.714467,99.249863,97.703178,91.721918
min,2024-01-01 00:00:00,5.510000,3.500000,8.560000,20.000000,0.000000,-14.140000,-3.390000,0.615210,0.000000,0.000000,1.000000,1.000000,48.857143,57.666667,0.000000,0.000000,0.000000
25%,2024-04-01 00:00:00,15.480000,25.660000,25.542500,65.000000,82.000000,-2.010000,8.390000,1.398910,0.000000,1.000000,4.000000,13.000000,91.142857,92.900000,82.000000,81.000000,76.000000
50%,2024-07-01 00:00:00,24.935000,39.750000,39.950000,110.000000,100.000000,0.020000,13.575000,1.636788,0.000000,3.000000,7.000000,26.000000,99.714286,100.220833,99.000000,99.000000,97.000000
75%,2024-09-30 00:00:00,37.560000,59.600000,59.360000,155.000000,117.000000,2.040000,23.730000,1.837247,0.000000,5.000000,9.000000,39.000000,108.142857,106.658333,117.000000,117.000000,116.000000
max,2024-12-30 00:00:00,49.210000,94.870000,98.330000,199.000000,240.000000,12.240000,46.840000,2.652235,1.000000,6.000000,12.000000,52.000000,166.000000,166.000000,240.000000,240.000000,240.000000
std,NaN,12.356771,21.544111,21.666343,52.069501,28.857532,3.134252,10.714544,0.263536,0.231718,2.003460,3.444068,15.047322,12.827578,8.925050,29.276295,31.687226,39.018526


In [50]:
df.to_csv("D:/Code_ML/dynamic_pricing_engine/data/processed/processed_data.csv", index=False)

In [ ]:
#EOF